# Brusselator PDE

The Brusselator PDE is defined as follows:

$$
\begin{align}
\frac{\partial u}{\partial t} &= 1 + u^2v - 4.4u + \alpha (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}) + f(x, y, t) \\
\frac{\partial v}{\partial t} &= 3.4u - u^2 v + \alpha (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2})
\end{align}
$$

where

$$
f(x, y, t) = 
\begin{cases}
5 \qquad \text{if} (x - 0.3)^2 + (y - 0.6)^2 \leq 0.1^2 \ and \  t \geq 1.1  \\
0 \qquad \text{otherwise}
\end{cases}
$$

and the initial conditions are

$$
\begin{align}
u(x, y, 0) &= 22(y(1-y))^{1.5} \\
v(x, y, 0) &= 27(x(1-x))^{1.5}
\end{align}
$$

with the periodic boundary condition

$$
\begin{align}
u(x+1, y, 0) &= u(x, y, t)  \\
u(x, y+1, 0) &= u(x, y, t)
\end{align}
$$

on a timespan of $t \in [0, 11.5]$.

In [1]:
using ModelingToolkit
using MethodOfLines
using DifferentialEquations
using DomainSets
using Plots

In [2]:
@parameters x y t
@variables u(..) v(..)

Dt = Differential(t)
Dx = Differential(x)
Dy = Differential(y)
Dxx = Differential(x)^2
Dyy = Differential(y)^2

# Laplacian
∇²(u) = Dxx(u) + Dyy(u)

∇² (generic function with 1 method)

In [3]:
brusselator_f(x, y, t) = (((x-0.3)^2 + (y-0.6)^2) <= 0.1^2) * (t >= 1.1) * 5

brusselator_f (generic function with 1 method)

In [4]:
x_min = y_min = t_min = 0.0
x_max = y_max = 1.0
t_max = 11.5
α = 10.0

u0(x,y,t) = 22(y*(1-y))^(3/2)
v0(x,y,t) = 27(x*(1-x))^(3/2)

eq = [Dt(u(x,y,t)) ~ 1. + v(x,y,t)*u(x,y,t)^2 - 4.4*u(x,y,t) + α*∇²(u(x,y,t)) + brusselator_f(x, y, t),
      Dt(v(x,y,t)) ~ 3.4*u(x,y,t) - v(x,y,t)*u(x,y,t)^2 + α*∇²(v(x,y,t))]

2-element Vector{Equation}:
 Differential(t)(u(x, y, t)) ~ 1.0 + 10.0Differential(x)(Differential(x)(u(x, y, t))) + 10.0Differential(y)(Differential(y)(u(x, y, t))) + (u(x, y, t)^2)*v(x, y, t) + 5(t >= 1.1)*(((x - 0.3)^2 + (y - 0.6)^2) <= 0.010000000000000002) - 4.4u(x, y, t)
 Differential(t)(v(x, y, t)) ~ 10.0Differential(x)(Differential(x)(v(x, y, t))) + 10.0Differential(y)(Differential(y)(v(x, y, t))) + 3.4u(x, y, t) - (u(x, y, t)^2)*v(x, y, t)

In [5]:
domains = [x ∈ Interval(x_min, x_max),
           y ∈ Interval(y_min, y_max),
           t ∈ Interval(t_min, t_max)]

3-element Vector{Symbolics.VarDomainPairing}:
 Symbolics.VarDomainPairing(x, 0.0..1.0)
 Symbolics.VarDomainPairing(y, 0.0..1.0)
 Symbolics.VarDomainPairing(t, 0.0..11.5)

In [6]:
# Periodic BCs
bcs = [u(x,y,0) ~ u0(x,y,0),
       u(0,y,t) ~ u(1,y,t),
       u(x,0,t) ~ u(x,1,t),

       v(x,y,0) ~ v0(x,y,0),
       v(0,y,t) ~ v(1,y,t),
       v(x,0,t) ~ v(x,1,t)] 

6-element Vector{Equation}:
 u(x, y, 0) ~ 22.0(y^1.5)*((1 - y)^1.5)
 u(0, y, t) ~ u(1, y, t)
 u(x, 0, t) ~ u(x, 1, t)
 v(x, y, 0) ~ 27.0(x^1.5)*((1 - x)^1.5)
 v(0, y, t) ~ v(1, y, t)
 v(x, 0, t) ~ v(x, 1, t)

In [7]:
@named pdesys = PDESystem(eq, bcs, domains, [x, y, t], [u(x,y,t), v(x,y,t)])

PDESystem
Equations: Equation[Differential(t)(u(x, y, t)) ~ 1.0 + 10.0Differential(x)(Differential(x)(u(x, y, t))) + 10.0Differential(y)(Differential(y)(u(x, y, t))) + (u(x, y, t)^2)*v(x, y, t) + 5(t >= 1.1)*(((x - 0.3)^2 + (y - 0.6)^2) <= 0.010000000000000002) - 4.4u(x, y, t), Differential(t)(v(x, y, t)) ~ 10.0Differential(x)(Differential(x)(v(x, y, t))) + 10.0Differential(y)(Differential(y)(v(x, y, t))) + 3.4u(x, y, t) - (u(x, y, t)^2)*v(x, y, t)]
Boundary Conditions: Equation[u(x, y, 0) ~ 22.0(y^1.5)*((1 - y)^1.5), u(0, y, t) ~ u(1, y, t), u(x, 0, t) ~ u(x, 1, t), v(x, y, 0) ~ 27.0(x^1.5)*((1 - x)^1.5), v(0, y, t) ~ v(1, y, t), v(x, 0, t) ~ v(x, 1, t)]
Domain: Symbolics.VarDomainPairing[Symbolics.VarDomainPairing(x, 0.0..1.0), Symbolics.VarDomainPairing(y, 0.0..1.0), Symbolics.VarDomainPairing(t, 0.0..11.5)]
Dependent Variables: Num[u(x, y, t), v(x, y, t)]
Independent Variables: Num[x, y, t]
Parameters: SciMLBase.NullParameters()
Default Parameter ValuesDict{Any, Any}()

In [8]:
# Discretization
N = 16

dx = (x_max - x_min) / N
dy = (y_max - y_min) / N

order = 2

discretization = MOLFiniteDifference([x=>dx, y=>dy], t, approx_order=order, grid_align=center_align)

MOLFiniteDifference{MethodOfLines.CenterAlignedGrid}(Pair{Num, Float64}[x => 0.0625, y => 0.0625], t, 2, MethodOfLines.UpwindScheme(), MethodOfLines.CenterAlignedGrid(), Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())

In [9]:
# Discretize the PDESystem to an ODEProblem or NonlinearProblem.
prob = discretize(pdesys, discretization)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 11.5)
u0: 512-element Vector{Float64}:
 0.31203234967003113
 0.31203234967003113
 0.31203234967003113
 0.31203234967003113
 0.31203234967003113
 0.31203234967003113
 0.31203234967003113
 0.31203234967003113
 0.31203234967003113
 0.31203234967003113
 0.31203234967003113
 0.31203234967003113
 0.31203234967003113
 ⋮
 2.688734070638931
 3.063590342214851
 3.296208237393214
 3.374999999999999
 3.296208237393214
 3.063590342214851
 2.688734070638931
 2.1921268033293604
 1.6054645733187292
 0.9766542925609523
 0.3829487927768564
 0.0

In [10]:
# Solvers: https://diffeq.sciml.ai/stable/solvers/ode_solve/
sol = solve(prob, QNDF(), saveat=0.1)

retcode: Success
Interpolation: 1st order linear
t: 116-element Vector{Float64}:
  0.0
  0.1
  0.2
  0.3
  0.4
  0.5
  0.6
  0.7
  0.8
  0.9
  1.0
  1.1
  1.2
  ⋮
 10.4
 10.5
 10.6
 10.7
 10.8
 10.9
 11.0
 11.1
 11.2
 11.3
 11.4
 11.5
u: 116-element Vector{Vector{Float64}}:
 [0.31203234967003113, 0.31203234967003113, 0.31203234967003113, 0.31203234967003113, 0.31203234967003113, 0.31203234967003113, 0.31203234967003113, 0.31203234967003113, 0.31203234967003113, 0.31203234967003113  …  3.296208237393214, 3.374999999999999, 3.296208237393214, 3.063590342214851, 2.688734070638931, 2.1921268033293604, 1.6054645733187292, 0.9766542925609523, 0.3829487927768564, 0.0]
 [1.5238955945215658, 1.5238955837006296, 1.5238955284546636, 1.5238954449778006, 1.5238953538779003, 1.5238952741393572, 1.5238952203171003, 1.523895201373532, 1.5238952203171006, 1.5238952741393572  …  2.0249522709137686, 2.0249523136847047, 2.0249522709137686, 2.0249521479734076, 2.0249519606395463, 2.024951733978853, 2.02495

In [11]:
grid = get_discrete(pdesys, discretization)
discrete_x = grid[x]
discrete_y = grid[y]
discrete_t = sol[t]

solu = [map(d -> sol[d][i], grid[u(x, y, t)]) for i in 1:length(sol[t])]
solv = [map(d -> sol[d][i], grid[v(x, y, t)]) for i in 1:length(sol[t])]

116-element Vector{Matrix{Float64}}:
 [0.0 0.0 … 0.0 0.0; 0.3829487927768564 0.3829487927768564 … 0.3829487927768564 0.3829487927768564; … ; 0.3829487927768564 0.3829487927768564 … 0.3829487927768564 0.3829487927768564; 0.0 0.0 … 0.0 0.0]
 [0.0 2.024951148707828 … 2.0249511487078267 2.024951103260173; 2.024951154083223 2.024951132436187 … 2.0249511324361884 2.024951154083223; … ; 2.0249511540832237 2.0249511324361853 … 2.0249511324361897 2.0249511540832237; 2.024951103260173 2.024951148707828 … 2.0249511487078267 2.024951103260173]
 [0.0 2.081318762201143 … 2.081318762201209 2.081313929648186; 2.0813142680904364 2.081310345192691 … 2.0813103451928052 2.0813142680904364; … ; 2.081314268090454 2.081310345192746 … 2.0813103451927284 2.081314268090454; 2.081313929648186 2.081318762201143 … 2.081318762201209 2.081313929648186]
 [0.0 2.15045253447897 … 2.1504525344789913 2.150452274162384; 2.150452369545147 2.1504491429897032 … 2.1504491429895607 2.150452369545147; … ; 2.150452369545186 2.15

In [12]:
umax = maximum(maximum, solu)
vmax = maximum(maximum, solu)

4.1652038158631015

In [13]:
using Plots

anim = @animate for k in 1:length(discrete_t)
    heatmap(solu[k][2:end, 2:end], title="u @ t=$(discrete_t[k])", clims = (0.0, 4.5)) # 2:end since end = 1, periodic condition
end

mp4(anim, fps = 8)

┌ Info: Saved animation to 
│   fn = /tmp/cirrus-ci-build/docs/tmp.mp4
└ @ Plots /root/.julia/packages/Plots/SkUg1/src/animation.jl:126


Plots.AnimatedGif("/tmp/cirrus-ci-build/docs/tmp.mp4")

In [14]:
anim = @animate for k in 1:length(discrete_t)
    heatmap(solv[k][2:end, 2:end], title="v @ t=$(discrete_t[k])", clims = (0.0, 4.5))
end

mp4(anim, fps = 8)

┌ Info: Saved animation to 
│   fn = /tmp/cirrus-ci-build/docs/tmp.mp4
└ @ Plots /root/.julia/packages/Plots/SkUg1/src/animation.jl:126


Plots.AnimatedGif("/tmp/cirrus-ci-build/docs/tmp.mp4")